In [2]:
import torch
import os
import torch.nn.functional as F
from torch_geometric.data import Dataset, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import MNIST # Used for visualization later

In [3]:
class GNN_MNIST(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GNN_MNIST, self).__init__()
        self.conv1 = GCNConv(num_node_features, 32)
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, 128)
        
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # GNN layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)

        # Pooling Layer
        x = global_mean_pool(x, batch)

        # Classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=-1)

In [4]:

class MNISTGraphDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        # The parent class constructor handles everything.
        super(MNISTGraphDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        # This dynamically finds all data files in the processed folder.
        # It assumes you have 60,000 files from the training set.
        return [f'data_{i}.pt' for i in range(60000)]

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        # The base class automatically knows where to find this file.
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'), weights_only=False)
        return data

In [5]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in tqdm(loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

In [6]:
def test(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in tqdm(loader, desc="Testing"):
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

In [7]:
if __name__ == '__main__':
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load dataset

    dataset = MNISTGraphDataset(root='./mnist_graphs') # <-- Use the new root path
    
    # Split dataset into training and testing
    # Note: A fixed random_state ensures the split is the same every time.
    train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
    train_dataset = dataset[train_indices]
    test_dataset = dataset[test_indices]
    
    print(f"Number of training graphs: {len(train_dataset)}")
    print(f"Number of test graphs: {len(test_dataset)}")

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,num_workers=0,pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,num_workers=0,pin_memory=True)

    # Initialize model, optimizer
    # Node features are [intensity, pos_y, pos_x], so num_node_features=3
    model = GNN_MNIST(num_node_features=3, num_classes=10).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 50
    history = {'loss': [], 'accuracy': []}
    
    for epoch in range(1, epochs + 1):
        loss = train(model, train_loader, optimizer, device)
        test_acc = test(model, test_loader, device)
        history['loss'].append(loss)
        history['accuracy'].append(test_acc)
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')


print("Saving trained model state...")
torch.save(model.state_dict(), 'gnn_mnist_model.pth')
print("Model saved to gnn_mnist_model.pth")

Using device: cuda
Number of training graphs: 48000
Number of test graphs: 12000


c:\Program Files\Python313\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Testing: 100%|██████████| 94/94 [00:44<00:00,  2.11it/s]


Epoch: 01, Loss: 2.1160, Test Accuracy: 0.2552


Testing: 100%|██████████| 94/94 [00:06<00:00, 15.66it/s]


Epoch: 02, Loss: 1.9856, Test Accuracy: 0.2838


...

Epoch: 48, Loss: 0.6692, Test Accuracy: 0.8256


Testing: 100%|██████████| 94/94 [00:16<00:00,  5.83it/s]


Epoch: 49, Loss: 0.6634, Test Accuracy: 0.8190


Testing: 100%|██████████| 94/94 [00:13<00:00,  6.72it/s]


Epoch: 50, Loss: 0.6640, Test Accuracy: 0.8369
Saving trained model state...
Model saved to gnn_mnist_model.pth
